<a href="https://colab.research.google.com/github/AbhimanyuAryan/stipple-fastai/blob/main/Julia_Colab_Notebook_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-12-19 17:36:52 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing Julia package CUDA...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 2


In [3]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  487.459 ms … 615.036 ms  ┊ GC (min … max): 0.00% … 18.97%
 Time  (median):     498.145 ms               ┊ GC (median):    0.10%
 Time  (mean ± σ):   520.232 ms ±  47.599 ms  ┊ GC (mean ± σ):  4.59% ±  7.25%

  █▁█    ▁▁        ▁                                    ▁     ▁  
  ███▁▁▁▁██▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█ ▁
  487 ms           Histogram: frequency by time          615 ms <

 Memory estimate: 32.00 MiB, allocs estimate: 2.

In [4]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

BenchmarkTools.Trial: 699 samples with 1 evaluation.
 Range (min … max):  5.966 ms …  10.239 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.181 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.144 ms ± 316.134 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                     ▁▃▂▂    ▁    ▄█▃ ▁        
  ▃▂▂▂▂▁▁▁▁▁▃▂▃▃▃▂▁▂▂▁▁▂▁▁▁▁▁▂▂▄▄▄▅▄▆████▆▄▄▆█▇▇▅▇███▆█▄▃▃▃▂▃ ▃
  5.97 ms         Histogram: frequency by time        7.61 ms <

 Memory estimate: 208 bytes, allocs estimate: 8.

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [5]:
using Pkg

In [6]:
Pkg.add("FastAI")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed ImageShow ───────────────── v0.3.1
   Installed Crayons ─────────────────── v4.0.4
   Installed ZygoteRules ─────────────── v0.2.2
   Installed DiffRules ───────────────── v1.9.0
   Installed Distances ───────────────── v0.10.7
   Installed OnlineStats ─────────────── v1.5.13
   Installed IntelOpenMP_jll ─────────── v2018.0.3+2
   Installed ThreadPools ─────────────── v1.2.1
   Installed Netpbm ──────────────────── v1.0.1
   Installed MKL_jll ─────────────────── v2021.1.1+2
   Installed FFTViews ────────────────── v0.3.2
   Installed Images ──────────────────── v0.24.1
   Installed Zygote ──────────────────── v0.6.32
   Installed SimpleTraits ────────────── v0.9.4
   Installed ArnoldiMethod ───────────── v0.1.0
   Installed ImageMagick_jll ─────────── v6.9.10-12+3
   Installed Media ───────────────────── v0.5.0
   Installed WoodburyMatrices ────────── v0.5.5
   Installed Calculus ──────

In [7]:
using FastAI

In [ ]:
data, blocks = loaddataset("imagenette2-160", (Image, Label))
method = ImageClassificationSingle(blocks)
learner = methodlearner(method, data, callbacks=[ToGPU()])
fitonecycle!(learner, 10)
showoutputs(method, learner)

This program has requested access to the data dependency fastai-imagenette2-160.
which is not currently installed. It can be installed automatically, and you will not see this message again.

"imagenette2-160" from the fastai dataset repository (https://course.fast.ai/datasets)

A subset of 10 easily classified classes from Imagenet: tench, English springer, cassette player, chain saw, church, French horn, garbage truck, gas pump, golf ball, parachute

Download size: ???



Do you want to download the dataset from https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz to "/root/.julia/datadeps/fastai-imagenette2-160"?
[y/n]


┌ Info: Downloading
│   source = https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz
│   dest = /root/.julia/datadeps/fastai-imagenette2-160/imagenette2-160.tgz
│   progress = 1.0
│   time_taken = 2.13 s
│   time_remaining = 0.0 s
│   average_speed = 44.327 MiB/s
│   downloaded = 94.417 MiB
│   remaining = 0 bytes
│   total = 94.417 MiB
└ @ HTTP /root/.julia/packages/HTTP/aTjcj/src/download.jl:131



7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)


Extracting archive: 
--
Path = 
Type = tar
Code Page = UTF-8

Everything is Ok

Folders: 23
Files: 13397
Size:       107794109
Compressed: 6872064


Epoch 1 TrainingPhase(): 100%|██████████████████████████| Time: 0:06:54


┌───────────────┬───────┬─────────┐
│         Phase │ Epoch │    Loss │
├───────────────┼───────┼─────────┤
│ TrainingPhase │   1.0 │ 2.13972 │
└───────────────┴───────┴─────────┘


...

Epoch 7 TrainingPhase(): 100%|██████████████████████████| Time: 0:03:24


┌───────────────┬───────┬────────┐
│         Phase │ Epoch │   Loss │
├───────────────┼───────┼────────┤
│ TrainingPhase │   7.0 │ 1.1956 │
└───────────────┴───────┴────────┘


Epoch 7 ValidationPhase(): 100%|████████████████████████| Time: 0:00:16


┌─────────────────┬───────┬─────────┐
│           Phase │ Epoch │    Loss │
├─────────────────┼───────┼─────────┤
│ ValidationPhase │   7.0 │ 15.9335 │
└─────────────────┴───────┴─────────┘


Epoch 8 TrainingPhase():  70%|██████████████████▍       |  ETA: 0:01:00